## **NTU OSS TGIFHacks #121**
# **An Introduction to Natural Language Processing**

In this notebook, we'll be training a language model on a very small text snippet. 

We will be using the GloVe embeddings as the initial weights for the model and train a simple bidirectional LSTM.



In [63]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, RNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.utils as ku 
import numpy as np 

!wget https://resources.oreilly.com/conferences/natural-language-processing-with-deep-learning/raw/master/data/glove.6B.100d.txt

--2021-01-28 04:34:40--  https://resources.oreilly.com/conferences/natural-language-processing-with-deep-learning/raw/master/data/glove.6B.100d.txt
Resolving resources.oreilly.com (resources.oreilly.com)... 199.27.144.213, 199.27.144.212
Connecting to resources.oreilly.com (resources.oreilly.com)|199.27.144.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘glove.6B.100d.txt.2’

glove.6B.100d.txt.2 100%[===================>] 331.04M  37.1MB/s    in 10s     

2021-01-28 04:34:51 (32.8 MB/s) - ‘glove.6B.100d.txt.2’ saved [347116733/347116733]



In [65]:
# Load the GloVe Eembddings
# This version of the embeddings has around 6 Billion words and each word has a word embeddings of dimension of 100

print('Indexing word vectors.')
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [66]:
tokenizer = Tokenizer()

# This is "Lanigan's Ball", its a popular fold irish song
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."
corpus = data.lower().split('\n')

# Train the tokenizer on the text
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
input_sequences = []

for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram = token_list[:i+1]
    input_sequences.append(n_gram)

In [67]:
'''
Pad all sentences to max length.

Padding Type: Pre
[0 0 0 0 0 4 5 6 6 23 3]

Padding Type: Post
[4 5 6 6 23 3 0 0 0 0 0]

'''

max_seq_len = max([len(x) for x in input_sequences])
padded_seq = np.array(pad_sequences(input_sequences, maxlen= max_seq_len, padding='pre')) 

xs, labels = padded_seq[:, :-1], padded_seq[:, -1]

#labels need to be converted class encodings
ys = ku.to_categorical(labels, num_classes=total_words)


In [68]:
'''
This is initializes the weights matrix for all the words in the vocab and the glove embeddings
'''
embedding_matrix = np.zeros((total_words, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(263, 100)


In [69]:
model = Sequential([
	Embedding(total_words, 100, input_length=max_seq_len-1, weights=[embedding_matrix]),
	Bidirectional(LSTM(20)),
	Dense(total_words, activation='softmax')
])

In [70]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
15/15 [==============================] - 3s 8ms/step - loss: 5.5585 - accuracy: 0.0026
Epoch 2/100
15/15 [==============================] - 0s 7ms/step - loss: 5.4939 - accuracy: 0.0419
Epoch 3/100
15/15 [==============================] - 0s 6ms/step - loss: 5.4127 - accuracy: 0.0715
Epoch 4/100
15/15 [==============================] - 0s 7ms/step - loss: 5.2844 - accuracy: 0.0610
Epoch 5/100
15/15 [==============================] - 0s 7ms/step - loss: 5.1412 - accuracy: 0.0782
Epoch 6/100
15/15 [==============================] - 0s 7ms/step - loss: 5.0386 - accuracy: 0.0690
Epoch 7/100
15/15 [==============================] - 0s 6ms/step - loss: 4.8727 - accuracy: 0.0816
Epoch 8/100
15/15 [==============================] - 0s 6ms/step - loss: 4.8737 - accuracy: 0.0665
Epoch 9/100
15/15 [==============================] - 0s 7ms/step - loss: 4.8271 - accuracy: 0.0624
Epoch 10/100
15/15 [==============================] - 0s 7ms/step - loss: 4.7400 - accuracy: 0.0771
Epoch 11/

In [71]:
seed_text = "you're my only hope"
next_words = 10
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


you're my only hope were all for chaneys for nice at all entangled groups


You can clearly observe that the performance of the model is not that great, its because of the small size of the training data. Usually language models are trained on more than a billions word corpus. 

I hope this gives a very basic idea about how language modeling task is actually implemented.